In [14]:
# Import Library
import pandas as pd
import os
from dotenv import load_dotenv
import pymongo
from pymongo import MongoClient
from langchain_openai import ChatOpenAI
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.chat_models import ChatOpenAI
import re

In [15]:
# Load semua environment
load_dotenv()
MONGODB_URI = os.getenv("MONGODB_URI")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
DEEPSEEK_API_NEW=os.getenv("DEEPSEEK_API_NEW")

In [16]:
# Embeding menggunakan gemini
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=GOOGLE_API_KEY,
)

In [17]:
# MongoDB Connection ke
client = MongoClient(MONGODB_URI)
collection = client['finalproject_db']['faq_300']

In [18]:
# Vector Store Configuration
vector_store = MongoDBAtlasVectorSearch(
    collection=collection,
    embedding=embeddings,
    index_name='vector_index',
)

In [ ]:
# Prompt Template
PROFESSIONAL_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    Anda adalah Asisten Virtual Resmi BPJS Indonesia.

    Peran Anda:
    Anda bertindak sebagai Customer Service (Layanan Pelanggan) untuk membantu masyarakat dalam memahami layanan dan prosedur dari:
    - Website BPJS Kesehatan
    - FAQ BPJS Ketenagakerjaan
    - FAQ Aplikasi JKN Mobile

    Tugas & Tanggung Jawab:
    1. Memberikan jawaban akurat, terpercaya, dan relevan sesuai dengan data dari sumber resmi (FAQ).
    2. Menjelaskan prosedur atau solusi dengan cara yang mudah dimengerti, terstruktur, dan menggunakan Bahasa Indonesia baku.
    3. Menjadi perantara informasi yang ramah, efisien, dan profesional dalam menangani keluhan atau pertanyaan pengguna.
    4. Jawablah dengan ringkas, jangan menjawab hal yang tidak ditanyakan
    5. Pertanyaan mungkin menggunakan bahasa tidak baku atau typo, sehingga anda harus paham perbedaannya dengan konteks yang berada diluar FAQ.
    7. Periksalah apakah pertanyaan merupakan suatu singkatan, yang mungkin terdapat jawabannya di dataset.

    Aturan yang Wajib Dipatuhi:
    1. Jawab hanya berdasarkan informasi yang tersedia dalam dokumen FAQ resmi.
    2. Jangan mengarang jawaban atau memberikan informasi tambahan yang tidak tercantum dalam sumber resmi.
    3. Jika pertanyaan tidak relevan atau tidak tersedia dalam data, sampaikan secara sopan bahwa Anda tidak memiliki informasi tersebut.
    4. Dilarang mencantumkan tautan eksternal, nomor telepon, alamat, atau informasi pribadi lain yang tidak ada dalam dokumen FAQ, jika ada diperbolehkan.
    5. Gunakan:
    - Penomoran (1., 2., 3., dst) untuk menjelaskan langkah atau prosedur berurutan.
    - Simbol bullet (•) untuk daftar yang tidak berurutan.

    Konteks resmi:
    {context}

    Pertanyaan: {question}

    Jawaban profesional:
    """
)

In [20]:
# Config model untuk Deepseek
llm_deepseek = ChatOpenAI(
    model="deepseek-chat",  
    openai_api_key=DEEPSEEK_API_NEW,  
    base_url="https://api.deepseek.com/v1",  
    temperature=0,
    max_tokens=500
)

In [21]:
# Retrieval Chain
qa = RetrievalQA.from_chain_type(
    llm=llm_deepseek,
    chain_type="stuff",
    retriever=vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={
            "k": 5,
            "score_threshold": 0.85
        }
    ),
    chain_type_kwargs={"prompt": PROFESSIONAL_PROMPT},
)

In [22]:
# Ambil data
faq_df = pd.read_csv('../data/final_dataset_clean.csv')
def answer_from_doc(question):
    """Fungsi untuk mencocokkan pertanyaan dengan data yang sama persis dan mengambil jawaban"""
    row = faq_df[faq_df['question'] == question]
    if not row.empty:
        return row.iloc[-1]['answer']
    else:
        return "Tidak ada jawaban."

In [23]:
def clean_answer(text):
    """Hilangkan tanda bold/italic Markdown"""
    text = re.sub(r'\*\*(.*?)\*\*', r'\1', text)
    text = re.sub(r'\*(.*?)\*', r'\1', text)
    cleaned_text = re.sub(r'\n{3,}', '\n\n', text)
    return cleaned_text.strip()
    

In [24]:
# UJI COBA 1: Pertanyaan yang Seharusnya Bisa Dijawab karena berasal dari dataset
questions = [
# Mobile
"Bagaimana cara menginstall aplikasi Jamsostek Mobile (JMO)?",
"Bagaimana cara mengajukan klaim JHT melalui JMO jika memiliki lebih dari 1 kartu?",
"Promo apa saja yang tersedia pada aplikasi Jamsostek Mobile (JMO)?",
"Jika anggota baru di JMO, apakah harus mendaftar di MotionPay?",
"Apakah itu Promo?",
"Apakah itu MotionPay?",
"Apakah sandi/PIN harus diubah secara berkala di kedua aplikasi?",
"Promo apa saja yang tersedia pada aplikasi Jamsostek Mobile (JMO)?",

# Ketenagakerjaan
"Apa itu BPJS Ketenagakerjaan?",
"Apa saja program BPJS Ketenagakerjaan?",
"Siapakah yang dimaksud peserta BPJS Ketenagakerjaan?",
"Apakah yang dimaksud dengan PHK?",
"Apakah pegawai BHL (Buruh Harian Lepas) dapat mengikuti program JKP?",
"Bagaimana caranya badan usaha/pemberi kerja dapat menggunakan layanan Payment Reminder System(PRS)?",
"Peserta BPU sudah melakukan pembayaran iuran namun transaksi pembayaran iuran belum merubah masa perlindungan/saldo JHT belum bertambah, apakah yang harus dilakukan?",
"Apakah aplikasi Electronic Payment System (EPS)?",
"Apa perbedaan program Jaminan Hari Tua (JHT) dan Jaminan Pensiun (JP)?",

# Kesehatan
"Apa saja pelayanan BPJS Kesehatan yang dapat dimanfaatkan ?",
"Bagaimana Mendaftarkan Bayi Baru Lahir ?",
"Dimana kontak kantor BPJS Kesehatan ?",
"Apa itu GRC ?",
"Cara Membayar Iuran",
"Apa saja pelayanan BPJS Kesehatan yang dapat dimanfaatkan ?",
"Kapan pembayaran iuran paling lambat ?",
"Bagaimana komitmen BPJS Kesehatan terkait gratifikasi ?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    result = qa.invoke({"query": q})
    print(clean_answer(result['result']))
    print(f"\nACTUAL DATA: {answer_from_doc(q)}")
    print(f"{'='*50}")


Pertanyaan: Bagaimana cara menginstall aplikasi Jamsostek Mobile (JMO)?
Untuk menginstall aplikasi JKN Mobile (sebelumnya dikenal sebagai Jamsostek Mobile/JMO), berikut langkah-langkahnya:  

1. Buka Play Store (Android) atau App Store (iOS) di perangkat Anda.  
2. Cari "JKN Mobile" di kolom pencarian.  
3. Pilih aplikasi resmi yang diterbitkan oleh BPJS Kesehatan.  
4. Klik "Install" dan tunggu proses pengunduhan selesai.  
5. Buka aplikasi setelah terinstall.  

Pastikan perangkat Anda mendukung sistem operasi terbaru dan memiliki ruang penyimpanan yang cukup.  

Jika pertanyaan lain, silakan hubungi layanan pelanggan BPJS Kesehatan melalui saluran resmi.

ACTUAL DATA: Aplikasi Jamsostek Mobile (JMO) dapat diunduh dan diinstall melalui Playstore/ Appstore bagi pengguna smartphone .

Pertanyaan: Bagaimana cara mengajukan klaim JHT melalui JMO jika memiliki lebih dari 1 kartu?
Untuk mengajukan klaim Jaminan Hari Tua (JHT) melalui JMO (Jamsostek Mobile) jika memiliki lebih dari 1 kartu

Total pertanyaan sesuai dataset : 25

Pertanyaan terjawab             : 18

Pertanyaan tidak terjawab       : 7

In [25]:
# # UJI COBA 2: Pertanyaan di Luar Konteks
# questions = [
#     "Apa itu Allo Bank?",
#     "Bagaimana cara mengajukan KPR?",
#     "Siapa penemu listrik?",
#     "Apa saja manfaat dari olahraga?",
#     "Bagaimana cara membuat kue?",
#     "Jelaskan apa itu Quantum Computing?",
#     "Apa yang dimaksud dengan Machine Learning?",
#     "Gimana cara buat foto HD dengan AI?",
#     "Apa itu teknologi blockchain?",
#     "Apa itu teknologi quantum?",
#     "Apa kopi paling enak saat ini?",
#     "Sebutkan Sepatu lari paling enak",
#     "Kapan Indonesia Merdeka?",
#     "Dimana kantor urusan agama terdekat",
#     "Jelaskan tentang skema piala dunia?",
#     "Siapa pencetak gol terbanyak piala dunia 2022?",
#     "Jumlah total trofi Liverpool berapa?",
#     "Hitung luas Kota Surabaya",
#     "Dimana tempat syuting film laskar Pelangi?",
#     "Dimana bom atom amerika saat perang dunia kedua dibuat?",
#     "Bagaimana cara membedakan biji kopi?",
#     "Jembatan merah ada Dimana?",
#     "Siapa presiden Indonesia pertama?",
#     "Siapa penemu alat musik gitar?",
#     "Hitung luas lapangan bola."
#     ]

# for q in questions:
#     print(f"\n{'='*50}")
#     print(f"Pertanyaan: {q}")
#     result = qa.invoke({"query": q})
#     print(clean_answer(result['result']))
#     print(f"\nACTUAL DATA: {answer_from_doc(q)}")
#     print(f"{'='*50}")

Total pertanyaan diluat dataset : 25

Pertanyaan terjawab             : 0

Pertanyaan tidak terjawab       : 25

In [26]:
# # UJI COBA 3: Pertanyaan Masih dalam Konteks namun terdapat kesalahan input
# questions = [
#    "Aph itu BPJS Ketenagakerajaan?",
#     "Apa sajak program BPJS Ketenagakerjaan?",
#     "Siapakah yang dimaksud peserta BPJS Ketenagakerajaan?",
#     "Apakh yang dimaksud dengan PHK?",
#     "Apakah pegawai BHL (Burung Harian Lepas) dapat mengikuti program JKP?",
#     "Peserta BPU sudah melakukan pembayaran iuran namun transaksi pembayaran iuran belum merubah masa perlindungan/saldo JHT belum bertambah, apakah yang harus dilakukan?", 
#     "Gimana jika lupa PIN EPS?",
#     "Apakah itu Perumahan Pekerja?",
#     "Dokumen apa sajakah persyaratan mengajukan JHT bagi PMI yang berpindah menjadi Warga Negara Asing?",
#     "Kapan BSU tahun 2025 dibagikan?",
#     "Dimana obat batuk pertama kali ditemukan",
#     "Jika flu sebaiknya minum obat herbal apa",
#     "Sopo peserta BPJS ketenagakerjaan",
#     "Sejak tanggal berapa pembayaran iuran paling lambat ?",
#     "Jika anggota baru di JMO, apakah harus mendaftar di MotionIme?",
#     "Apakah itu ShoopePay?",
#     "Apa perbedaan program Jaminan Dari Tua (JHT) dan Jaminan Pensiun (JP)?",
#     "Gimana pembayaran iuran?",
#     "Tanggal pembayaran iuran paling lambat ?",
#     "Promo apa sajak yang tersedia pada aplikasi Jamsostek Mobile (JMO)?"
# ]

# for q in questions:
#     print(f"\n{'='*50}")
#     print(f"Pertanyaan: {q}")
#     result = qa.invoke({"query": q})
#     print(clean_answer(result['result']))
#     print(f"\nACTUAL DATA: {answer_from_doc(q)}")
#     print(f"{'='*50}")

Total pertanyaan salah tidak baku/typo : 20

Pertanyaan terjawab             : 12

Pertanyaan tidak terjawab       : 8